# FlatField Class [v1]

In [1]:
%matplotlib inline

In [79]:
# imports
import os
from importlib import reload
import numpy as np

from astropy.table import Table

from pypit import flatfield
from pypit import processimages  # Mainly for settings
from pypit.core import arprocimg

In [3]:
# Path to PYPIT-Development-suite
pypdev_path = os.getenv('PYPIT_DEV')

# User options

----

# Development

In [4]:
from pypit import traceslits
from pypit import wavetilts

## Load up required MasterFrames

In [16]:
settings = processimages.default_settings.copy()
settings['masters'] = {}
settings['masters']['directory'] = pypdev_path+'/Cooked/MF_shane_kast_blue'
settings['masters']['reuse'] = True
settings['masters']['loaded'] = []

In [6]:
setup = 'A_01_aa'

### TraceSlits

In [135]:
reload(traceslits)
TSlits = traceslits.TraceSlits.from_master_files(settings['masters']['directory']+'/MasterTrace_A_01_aa')
TSlits._make_pixel_arrays()
_ = TSlits._fill_slits_dict()

[INFO]    :: traceslits.py 207 from_master_files() - Loading BPM from /home/xavier/local/Python/PYPIT-development-suite//Cooked/MF_shane_kast_blue/MasterTrace_A_01_aa.fits.gz
[INFO]    :: traceslits.py 518 _make_pixel_arrays() - Converting physical trace locations to nearest pixel
[INFO]    :: traceslits.py 525 _make_pixel_arrays() - Identifying the pixels belonging to each slit


### Tilts

In [10]:
wvTilts = wavetilts.WaveTilts(None, settings=settings, setup=setup)
tilts = wvTilts.master()

[INFO]    :: armasters.py 241 _load() - Loading a pre-existing master calibration frame


### Detector settings (for datasec and gain)

In [42]:
settings['detector']['dataext'] = 0
settings['detector']['datasec01'] = [[0, 1024], [0, 0]]
settings['detector']['datasec02'] = [[1024, 2048], [0, 0]]
settings['detector']['oscansec01'] = [[2049, 2080], [0, 0]]
settings['detector']['oscansec02'] = [[2080, 2111], [0, 0]]
settings['detector']['naxis0'] = 2112  # Raw frame, with overscan
settings['detector']['naxis1'] = 350
settings['detector']['numamplifiers'] = 2
settings['detector']['gain'] = [1.2,1.2]

In [28]:
tilts.shape

(2048, 350)

In [29]:
reload(arprocimg)
datasec_img, naxis0, naxis1 = arprocimg.get_datasec_trimmed(
    'shane_kast_blue', None, settings['detector']['numamplifiers'], 1, settings['detector'],
    naxis0=settings['detector']['naxis0'], naxis1=settings['detector']['naxis1'])
datasec_img.shape

(2048, 350)

----

## Init FlatField

In [142]:
reload(flatfield)
ftField = flatfield.FlatField(spectrograph='shane_kast_blue', settings=settings, det=1,
                             tilts=tilts, slits_dict=TSlits.slits_dict.copy())

### Load mstrace

In [143]:
ftField.mspixelflat = TSlits.mstrace.copy()

In [144]:
ftField.show('mspixelflat')

### Apply gain

In [145]:
ftField.apply_gain(datasec_img)

### Normalize a slit

In [146]:
slit = 0

In [147]:
modvals, nrmvals, msblaze_slit, blazeext_slit, iextrap_slit = ftField.slit_profile(slit)

#### Apply

In [150]:
word = np.where(ftField.slits_dict['slitpix'] == slit+1)

In [151]:
ftField.mspixelflatnrm = ftField.mspixelflat.copy()
ftField.mspixelflatnrm[word] /= nrmvals

#### Show

In [98]:
ftField.show('norm')

## Towards the loop

In [118]:
ftField._prep_tck()
ftField.ntckx, ftField.ntcky

(614, 20)

----

# TODO
    Remove settings from norm_slit
    Polish
    Test build

----

# Scratch space

In [124]:
TSlits.pixwid

array([307])